In [ ]:
import re
import os
import shutil

from datetime import datetime

output_dir='../src/main/resources/

def snippet_filename(schema, table_name, version, counter):
    return '{}/{}-dml-{}-{:02d}.sql'.format(schema, table_name, version, counter)

In [ ]:
default_stmt = ['--liquibase formatted sql\n\n']

def create_snippets(schema, order):
    start = r"LOCK TABLES `(.*)` WRITE;"
    end = r"UNLOCK TABLES;"
    insert_stmt = r"INSERT INTO `(.*)` VALUES"
    
    file_name = '{}_dml.sql'.format(schema)
    
    output_files=[]
    
    with open(file_name, encoding='UTF-8') as f:
        table_order = 10
        counter = 0
        
        all_lines = f.readlines()

        table_name = None
        append=False
        
        stmt = default_stmt.copy()

        fk_stmt = []
        for line in all_lines:            
            m1 = re.search(start, line)
            m2 = re.search(end, line)
            m3 = re.search(insert_stmt, line)
            m4 = re.search(r"/*!40000 ALTER TABLE", line)
            
            skip = False
            
            if (m3):
                table_name = m3.group(1)
                #print(table_name)
                    
                if len(stmt) == 0:
                    stmt = default_stmt.copy()
                    
                if not append and not skip:
                    append = True
                    changeset_comment='--changeset bibagimon@gmail.com:{}.{}\n'.format(order, table_order)
                    stmt.append(changeset_comment)
                
                
            if append and not m2 and not m4:
                if m3:
                    line = re.sub(insert_stmt, "INSERT INTO `{}`.`{}` VALUES".format(schema, table_name), line)
                stmt.append(line)

                
            if append and m2 and table_name:
                append = False
                stmt.append('--rollback DELETE FROM `{}`.`{}`;\n'.format(schema, table_name))
                stmt.append('\n\n')
                
                output_file=snippet_filename(schema, table_name, '1.2.0', 1)
                output_files.append(output_file)
                #print(output_file)

                if not os.path.exists(output_dir + schema):
                    os.mkdir(output_dir + schema)

                with open(output_dir + output_file, 'w') as out:
                    out.writelines(stmt)
                    out.flush()

                table_name = None
                table_order += 1
                stmt = []
                
    return output_files

In [ ]:
schemas = ['bpm', 'product', 'reference']

all_output_files=[]
for i, schema in enumerate(schemas):
    all_output_files.append(create_snippets(schema, i+11))

In [ ]:
for item in all_output_files:
    for file in sorted(item):
        print('<include file="{}" relativeToChangelogFile="true"/>'.format(file))